In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from patsy import dmatrices
pd.options.mode.chained_assignment = None

from sklearn import preprocessing

from sklearn.metrics import accuracy_score, roc_curve, auc, classification_report, confusion_matrix

from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.learning_curve import learning_curve
from sklearn.grid_search import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import pprint

%matplotlib inline

In [ ]:
df = pd.read_csv("3355.csv")

###Condensing dataframe down to features to explore

In [ ]:
df2 = df[[
'CASENUM','YEAROB1','SEX','RACE','ETHNIC', 
'ADTYP','RELTYP','NFRCTNS',
'DRUGTRT','ALCTRT','SEXTRT','EDUCAT','VOCAT',
'SMPOFF5','RELAGE','SNTLN','TMSRV','PCTSRV',
'A001CNT', 'A001YR','A001MO','A001ST','A001FM1', 'A001OFF1',
'J001MO', 'J001YR', 'J001CNT', 'J001OFF1', 'J001FM1', 'J001CNV1', 'J001CNF1', 'J001PJP1', 'J001PMX1', 'J001PRB1',
'REARR','ROTST','PRIR','POTST']]
df2.head()

###Replacing unknown numeric values with nan (for now)

In [ ]:
df2['YEAROB1'] = df2['YEAROB1'].replace(9999,np.nan)
df2['A001YR'] = df2['A001YR'].replace(9999,np.nan)
df2['A001YR'] = df2['A001YR'].replace(9998,np.nan)
df2['REARR'] = df2['REARR'].replace(888,np.nan)
df2['ROTST'] = df2['ROTST'].replace(888,np.nan)
df2['PRIR'] = df2['PRIR'].replace(888,np.nan)
df2['POTST'] = df2['POTST'].replace(888,np.nan)
df2['RELAGE'] = df2['RELAGE'].replace(99999999.99,np.nan)
df2['RELAGE'] = df2['RELAGE'].replace(100000000.00,np.nan)
df2['TMSRV'] = df2['TMSRV'].replace(99899899.88,np.nan)
df2['TMSRV'] = df2['TMSRV'].replace(998999.00,np.nan)
df2['SNTLN'] = df2['SNTLN'].replace(99899899.88,np.nan)
df2['SNTLN'] = df2['SNTLN'].replace(998999.00,np.nan)
df2['PCTSRV'] = df2['PCTSRV'].replace(99899899.88,np.nan)
df2['PCTSRV'] = df2['PCTSRV'].replace(998999.00,np.nan)
df2['A001CNT'] = df2['A001CNT'].replace(99,np.nan)
df2['A001CNT'] = df2['A001CNT'].replace(98,np.nan)
df2['J001YR'] = df2['J001YR'].replace(9999,np.nan)

###Adding calculated features

In [ ]:
df2['AGE_FIRST_OFF'] = df2.A001YR - df2.YEAROB1
df2['AGE_FIRST_OFF'][df2['AGE_FIRST_OFF'] < 16] = np.nan
df2['TOT_PRARR'] = (df2['PRIR'] + df2['POTST'])
df2['CAREER_LEN'] = df2.RELAGE-df2.AGE_FIRST_OFF
df2['SNTLN_YRS'] = df2.SNTLN/12
df2['TOTREARR'] = df2.REARR + df2.ROTST
df2['AVG_YRLY_ARR'] = df2.TOT_PRARR/df2.CAREER_LEN
df2['AVG_YRLY_ARR'][df2['AVG_YRLY_ARR'] < 0] = np.nan
df2['AVG_YRLY_ARR'][df2['AVG_YRLY_ARR'] > 100] = 0
df2['RESP'] = 0
df2['RESP'][df2['TOTREARR'] > 0] = 1

In [ ]:
treatments = ['DRUGTRT', 'ALCTRT', 'SEXTRT', 'EDUCAT', 'VOCAT']

for x in treatments:
    df2[x] = df2[x].replace('UNKNOWN',0)
    df2[x] = df2[x].replace('INMATE DID NOT PARTICIPATE',0)
    df2[x] = df2[x].replace('INMATE PARTICIPATED BUT UNKNOWN IF COMPLETED',1)
    df2[x] = df2[x].replace('INMATE PARTICIPATED IN PROGRAM & COMPLETED IT',1)
    df2[x] = df2[x].replace('INMATE PARTICIPATED BUT DID NOT COMPLETE',1)

df2['TREATMENT'] = df2.DRUGTRT + df2.ALCTRT + df2.SEXTRT + df2.EDUCAT + df2.VOCAT
for x in treatments:
    df2.drop(x, axis=1, inplace=True)
df2['TREATMENT'][df2['TREATMENT'] > 0] = 1

###Counting nan percentage

In [ ]:
features = df2.columns.values.tolist()

In [ ]:
for feature in features:
    per = float(float(len(df2[feature]) - df2[feature].count())/len(df2[feature]))
    if per > 0.0:
        print feature, "has %0.2f percent missing values" % (per*100)

###Cleaning up missing data

In [ ]:
#Hard coded the mode of the ROTST & POTST column
df2['ROTST']= df2.ROTST.replace(np.nan,0)
df2['POTST']= df2.POTST.replace(np.nan,0)

In [ ]:
df2['PRIR'] = df2.PRIR.replace(np.nan,df2.PRIR.mean())
df2['REARR'] = df2.REARR.replace(np.nan,df2.REARR.mean())
df2['PCTSRV'] = df2.PCTSRV.replace(np.nan,df2.PCTSRV.mean())
df2['TMSRV'] = df2.TMSRV.replace(np.nan,df2.TMSRV.mean())

###Recalculate features with cleaned data

In [ ]:
df2['AGE_FIRST_OFF'] = df2.A001YR - df2.YEAROB1
df2['AGE_FIRST_OFF'][df2['AGE_FIRST_OFF'] < 16] = np.nan
df2['TOT_PRARR'] = (df2['PRIR'] + df2['POTST'])
df2['CAREER_LEN'] = df2.RELAGE-df2.AGE_FIRST_OFF
df2['SNTLN_YRS'] = df2.SNTLN/12
df2['TOTREARR'] = df2.REARR + df2.ROTST
df2['AVG_YRLY_ARR'] = df2.TOT_PRARR/df2.CAREER_LEN
df2['AVG_YRLY_ARR'][df2['AVG_YRLY_ARR'] < 0] = np.nan
df2['AVG_YRLY_ARR'][df2['AVG_YRLY_ARR'] > 100] = 0
df2['RESP'] = 0
df2['RESP'][df2['TOTREARR'] > 0] = 1

###Recheck for missing values

In [ ]:
for feature in features:
    per = float(float(len(df2[feature]) - df2[feature].count())/len(df2[feature]))
    if per > 0.0:
        print feature, "has %0.2f percent missing values" % (per*100)

###Look at means for each group to be classified

In [ ]:
df2.groupby('RESP').mean()

###Quick visualize distribution of features

In [ ]:
def make_plot(x):
    plt.figure(figsize = (15,5))
    sns.countplot(sorted(df2[x]))
    plt.xticks(rotation = 90)
    plt.xlabel(x)
    plt.show()

for feature in features:
    make_plot(feature)

###Set up selected features, train/test split and scale

In [ ]:
y, X = dmatrices('RESP ~ ADTYP + np.log(A001CNT+1) + SEX + RACE  + ETHNIC +RELTYP + J001PJP1+ np.log(RELAGE+1) + np.log(TMSRV+1) + np.log(TOT_PRARR+1) + PCTSRV + np.log(AGE_FIRST_OFF) + SMPOFF5+ A001FM1 + C(TREATMENT) + SNTLN_YRS + np.log(AVG_YRLY_ARR + 1)', data=df2, return_type='dataframe')

y = np.ravel(y)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7)

std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_test = std_scale.transform(x_test)

###Baseline predictor


In [ ]:
def always_reoffend(x):
    return [1] * len(x)
y_pred = always_reoffend(x_test)

print "Baseline = %0.2f" % accuracy_score(y_test, y_pred) 

###Train model and evaluate test accuracy

In [ ]:
def test_model(name, main_model):
    model = main_model
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print name, "test accuracy score:", accuracy_score(y_test, model.predict(x_test))
    
test_model("KNN", KNeighborsClassifier())
test_model("Logistic Regression", LogisticRegression())
test_model("Gaussian Naive Bayes", GaussianNB())
test_model("SVM Classifier", SVC())
test_model("Decision Tree", DecisionTreeClassifier())
test_model("Random Forest", RandomForestClassifier())

###Look at feature importance

In [ ]:
model = RandomForestClassifier()
model.fit(x_train, y_train)
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

plt.figure(figsize = (25,10))
plt.title("Feature importances")
plt.bar(range(len(indices)), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(len(indices)), X.columns[indices], rotation = 90)
plt.xlim([-1, len(indices)])
plt.show()

###Create dataframe with confusion matrix metrics for each feature in model

In [ ]:
cm_features = ['AVG_YRLY_ARR','RELAGE','TOT_PRARR', 'TMSRV', 'AGE_FIRST_OFF','SNTLN_YRS', 'SMPOFF5','J001PJP1']

In [ ]:
model_list = []
for feature in cm_features:
    model = "y, X = dmatrices('RESP ~ " +feature+ "', data=df2, return_type='dataframe')"
    model_list.append(model)
pprint.pprint(model_list)

In [ ]:
y, X = dmatrices('RESP ~ ADTYP + np.log(A001CNT+1) + SEX + RACE  + ETHNIC +RELTYP + J001PJP1+ np.log(RELAGE+1) + np.log(TMSRV+1) + np.log(TOT_PRARR+1) + PCTSRV + np.log(AGE_FIRST_OFF) + SMPOFF5+ A001FM1 + C(TREATMENT) + SNTLN_YRS + np.log(AVG_YRLY_ARR + 1)', data=df2, return_type='dataframe')

y = np.ravel(y)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7)

std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_test = std_scale.transform(x_test)

model = SVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [ ]:
cm = confusion_matrix(y_test, model.predict(x_test))

In [ ]:
cm_df = pd.DataFrame(np.hstack((cm[0],cm[1])))
cm_df

In [ ]:
y, X = dmatrices('RESP ~ TMSRV', data=df2, return_type='dataframe')

y = np.ravel(y)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7)

std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_test = std_scale.transform(x_test)

model = SVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
cm = confusion_matrix(y_test, model.predict(x_test))
cm_df['TMSRV'] = np.hstack((cm[0],cm[1]))

In [ ]:
y, X = dmatrices('RESP ~ RELAGE', data=df2, return_type='dataframe')

y = np.ravel(y)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7)

std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_test = std_scale.transform(x_test)

model = SVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
cm = confusion_matrix(y_test, model.predict(x_test))
cm_df['RELAGE'] = np.hstack((cm[0],cm[1]))

In [ ]:
y, X = dmatrices('RESP ~ SNTLN_YRS', data=df2, return_type='dataframe')

y = np.ravel(y)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7)

std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_test = std_scale.transform(x_test)

model = SVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
cm = confusion_matrix(y_test, model.predict(x_test))
cm_df['SNTLN_YRS'] = np.hstack((cm[0],cm[1]))

In [ ]:
y, X = dmatrices('RESP ~ SMPOFF5', data=df2, return_type='dataframe')

y = np.ravel(y)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7)

std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_test = std_scale.transform(x_test)

model = SVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
cm = confusion_matrix(y_test, model.predict(x_test))
cm_df['SMPOFF5'] = np.hstack((cm[0],cm[1]))

In [ ]:
y, X = dmatrices('RESP ~ A001CNT', data=df2, return_type='dataframe')

y = np.ravel(y)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7)

std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_test = std_scale.transform(x_test)

model = SVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
cm = confusion_matrix(y_test, model.predict(x_test))
cm_df['A001CNT'] = np.hstack((cm[0],cm[1]))

In [ ]:
y, X = dmatrices('RESP ~ J001PJP1', data=df2, return_type='dataframe')

y = np.ravel(y)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7)

std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_test = std_scale.transform(x_test)

model = SVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
cm = confusion_matrix(y_test, model.predict(x_test))
cm_df['J001PJP1'] = np.hstack((cm[0],cm[1]))

In [ ]:
y, X = dmatrices('RESP ~ TOT_PRARR', data=df2, return_type='dataframe')

y = np.ravel(y)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7)

std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_test = std_scale.transform(x_test)

model = SVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
cm = confusion_matrix(y_test, model.predict(x_test))
cm_df['TOT_PRARR'] = np.hstack((cm[0],cm[1]))

In [ ]:
y, X = dmatrices('RESP ~ AVG_YRLY_ARR', data=df2, return_type='dataframe')
y = np.ravel(y)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7)

std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_test = std_scale.transform(x_test)

model = SVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
cm = confusion_matrix(y_test, model.predict(x_test))
cm_df['AVG_YRLY_ARR'] = np.hstack((cm[0],cm[1]))

In [ ]:
y, X = dmatrices('RESP ~ AGE_FIRST_OFF', data=df2, return_type='dataframe')

y = np.ravel(y)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7)

std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_test = std_scale.transform(x_test)

model = SVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
cm = confusion_matrix(y_test, model.predict(x_test))
cm_df['AGE_FIRST_OFF'] = np.hstack((cm[0],cm[1]))

In [ ]:
cm_df.columns =['Full model', 'Time served (1994)', 'Age at release (1994)' , 'Sentence Length (1994)','Severity of offense (1994)', 'Consequence (first arrest)', 'Total prior arrests (1994)','Average arrest rate','Age at first offense',]
cm_df =cm_df[['Full model', 'Time served (1994)', 'Age at release (1994)' , 'Sentence Length (1994)','Severity of offense (1994)','Consequence (first arrest)', 'Total prior arrests (1994)','Average arrest rate', 'Age at first offense']]
cm_df

In [ ]:
cm_plot_df = cm_df.T
cm_plot_df

In [ ]:
cm_plot_df['Correct']= cm_plot_df[0] + cm_plot_df[3]
cm_plot_df['Incorrect']= cm_plot_df[1] + cm_plot_df[2]
cm_plot_df['Type 1']= cm_plot_df[1]
cm_plot_df['Type 2']= cm_plot_df[2]
cm_plot_df = cm_plot_df[['Correct', 'Incorrect','Type 1','Type 2']]
cm_plot_df

In [ ]:
per_plot_df = cm_plot_df.copy()
per_plot_df['TOTAL'] = per_plot_df['Correct'] + per_plot_df['Incorrect']
per_plot_df['Correct'] = per_plot_df['Correct']/per_plot_df['TOTAL']
per_plot_df['Incorrect'] = per_plot_df['Incorrect']/per_plot_df['TOTAL']
per_plot_df = per_plot_df[['Correct', 'Incorrect']]
per_plot_df = per_plot_df.sort(['Correct'], ascending = False)
per_plot_df

In [ ]:
per_plot_df.to_csv('final_data.csv', sep=',', header = ['Correct', 'Incorrect'])

In [ ]:
in_per_plot_df = cm_plot_df.copy()
in_per_plot_df['TOTAL'] = in_per_plot_df['Correct'] + in_per_plot_df['Incorrect']
in_per_plot_df['Type 1'] = in_per_plot_df['Type 1']/in_per_plot_df['TOTAL']
in_per_plot_df['Type 2'] = in_per_plot_df['Type 2']/in_per_plot_df['TOTAL']
in_per_plot_df = in_per_plot_df[['Type 1', 'Type 2']]
in_per_plot_df = in_per_plot_df.sort(['Type 1'], ascending = False)
in_per_plot_df

In [ ]:
in_per_plot_df.to_csv('error_data.csv', sep=',', header = ['Innocent - presumed guilty', 'Guilty - presumed innocent'])

In [ ]:
def curves(name, classifier):
    probas_ = classifier.fit(x_train, y_train).predict_proba(x_test)
    fpr, tpr, thresholds = roc_curve(y_test, probas_[:, 1])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label = "%s area under curve = %0.2f" 
             % (name, roc_auc_score(y_test, probas_[:, 1])))
    plt.legend(loc = "best")
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    #plt.title("ROC curves")
    
curves("KNN", KNeighborsClassifier())
curves("Logistic Regression", LogisticRegression())
curves("Gaussian Naive Bayes", GaussianNB())
curves("SVM Classifier", SVC(probability = True))
curves("Decision Tree", DecisionTreeClassifier())
curves("Random Forest", RandomForestClassifier())

###Grid search for best parameters for SVC

In [ ]:
"""y, X = dmatrices('RESP ~ ADTYP + np.log(A001CNT+1) + SEX + RACE  + ETHNIC +RELTYP + J001PJP1+ np.log(RELAGE+1) + np.log(TMSRV+1) + np.log(TOT_PRARR+1) + PCTSRV + np.log(AGE_FIRST_OFF) + SMPOFF5+ A001FM1 + C(TREATMENT) + SNTLN_YRS + np.log(AVG_YRLY_ARR + 1)', data=df2, return_type='dataframe')

y = np.ravel(y)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7)

std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_test = std_scale.transform(x_test)

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)

    clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5,
                       scoring='%s_weighted' % score)
    clf.fit(x_train, y_train)

    print("Best parameters set found on development set:")
    print(clf.best_params_)
    print("Grid scores on development set:")
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))

    print("Detailed classification report:")
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    y_true, y_pred = y_test, clf.predict(x_test)
    print(classification_report(y_true, y_pred))"""